# Итоговый проект по автобрее
### Выполнили:
- Аксенова Анна
- Волошина Екатерина
- Кудрявцева Полина
- Такташева Екатерина

# Этап 1: Сбор данных

1. Корпус состоит из статей Википедии. 

2. Тексты фильтруются по некоторым паттернам, чтобы
    - избежать ошибки сегментации предложений
    - убрать предложения из списка литературы
    - удалить лишние заголовки, не несущие особого веса
    - удалить предложения с латинскими символами (потому что спасибо пайморфи)


3. Предложения токенизируются (spacy) и размечаются pymorphy

### Скачиваем дамп википедии:

In [1]:
!pip install --quiet corpuscula

In [8]:
from corpuscula import corpus_utils


# настройка каталога, в который скачается дамп википедии
corpus_path = "/Users/katya/УЧЕБА/Python/NLP/corpus/wiki"
corpus_utils.set_root_dir(corpus_path)

In [9]:
root_dir = corpus_utils.get_root_dir()

In [10]:
from corpuscula import wikipedia_utils


# скачивание википедии
wikipedia_utils.download_wikipedia(lang="RU", root_dir=root_dir, overwrite=True)

>########] 100%                                                    
done: 4097700193 bytes


'/Users/katya/УЧЕБА/Python/NLP/corpus/wiki/corpus/wikipedia_ru/ruwiki-latest-pages-articles.xml.bz2'

In [33]:
from corpuscula import wikipedia_utils


wik = wikipedia_utils.Wikipedia()

Грузим токенизатор:

In [52]:
from spacy.lang.ru import Russian
from spacy_russian_tokenizer import RussianTokenizer, MERGE_PATTERNS, SYNTAGRUS_RARE_CASES
nlp = Russian()
russian_tokenizer = RussianTokenizer(nlp, MERGE_PATTERNS + SYNTAGRUS_RARE_CASES)
nlp.add_pipe(russian_tokenizer, name='russian_tokenizer')

## Предобработка данных

Чистим от цитат:

In [ ]:
from rusenttokenize import ru_sent_tokenize
from string import punctuation

quotes = '«»”"' + "'"

def is_quote(sentence, quotes=quotes):
    """
    check if any quotation mark is in an input sentence
    """
    return True if any([quote in sentence for quote in quotes]) else False

От ошибок сегментации:

In [ ]:
def is_punct(sentence, punct_marks=punctuation+quotes):
    """
    check if some punctuation patterns are in an input sentence
    (gets rid of segmentation errors)
    """
    return True if any([sentence.startswith(punct_mark) for punct_mark in punct_marks]) else False

От предложений с латинскими словами:

In [ ]:
def is_latin(sentence):
    """
    checks if there is latin symbols in an input sentence
    """
    return True if re.match('[A-z]', sentence) else False

А теперь применим это все к данным:

In [536]:
def clean_article(a):
    """
    strips text of headers, subheaders, etc.
    """
    seg = [s for s in a.replace("\xa0", " ").strip().split("\n\n") if s and s.endswith(".")]
    _s = []
    for s in seg:
        _s.extend(s.split('\n'))
    _s = [s for s in _s if 5 <= len(s.split()) and s[0].isupper() and s.endswith(".")]
    return " ".join(_s)


def prepare_from_text(text):
    """
    text --> sentences
    filters sentences
    """
    res = []
    sentences = [s.strip() for s in ru_sent_tokenize(text)]
    for sentence in sentences:
        _is_quote = is_quote(sentence)
        _is_punct = is_punct(sentence)
        _is_latin = is_latin(sentence)
        if _is_quote or _is_punct or _is_latin:
            continue

        sent_in_words = [token.text for token in nlp(sentence) if token.text.isalpha()]
        if 5 <= len(sent_in_words):
            res.append(sentence)
    return res

## Скачиваем статьи

In [538]:
# инициализируем генератор статей
gen = wik.articles()

In [539]:
import os


print("I am here: %s" % os.getcwd())
c = 0

with open("wiki.json", "a", encoding="utf-8") as f:
    # индекс статьи (неупорядоченный), заголовок, текст статьи
    for idx, title, article in gen:
        c += 1
        # ограничимся 1000 текстами
        if c == 1000:
            break

        if not article or not title:
            continue

        a = clean_article(article)
        ares = prepare_from_text(a)
        
        temp = {
            "title": title,
            "res": ares
        }

        f.write(
            str(temp) + "\n"
        )

I am here: /Users/katya/УЧЕБА/Python/NLP


Process Wikipedia
[> 999                                                            

Загрузим скачанные данные:

In [540]:
import json

data = []
with open('wiki.json', 'r', encoding='utf-8') as fid:
    for line in tqdm(fid):
        text = json.loads(line.replace("'", '"'))
        data.append(text)

In [541]:
len(data)

901

### Почему мы выбрали Pymorphy?
Считаем, что нам больше важна полнота выдачи, чем ее точность, поэтому мы используем все варианты POS-тэгов, предлагаемые пайморфи.

**Приведем в порядок тэги:** немного обощим, убрав узкие группы, типа герундий (--> глагол), сранительная степень прилагательного (--> прилагательное) и т.д.

In [543]:
transform = {
    'NOUN': 'NOUN',
    'ADJF': 'ADJ',
    'ADJS': 'ADJ',
    'COMP': 'COMP',
    'VERB': 'VERB',
    'INFN': 'VERB',
    'PRTF': 'PTCP',
    'PRTS': 'PTCP',
    'GRND': 'VERB',
    'NUMR': 'NUM',
    'NPRO': 'PRON',
    'PRED': 'VERB',
    'PREP': 'PREP', 
    'CONJ': 'CONJ',
    'PRCL': 'PART',
    'INTJ': 'INTJ',
    'ADVB': 'AD

## Больше обработки:

In [ ]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [544]:
import re
from urllib.parse import quote  # чтобы были красивые ссылочки



def parse_analysis(analysis):
    """
    returns the analysis of the parsed word, 
    formatted like: lemma1|lemma2, POS1|POS2
    """
    tags = [(analysis[i].normal_form, 
             analysis[i].tag.POS) for i in range(len(analysis))]
    tags = list(set(tags))
    pos = [transform[t[1]] if t[1] is not None else 'UNK' for t in tags]
    lemmas = [t[0] if not None else 'UNK' for t in tags]
    return '|'.join(lemmas), '|'.join(pos)
    

def preprocess(sent):
    """
    returns tokenized text (without punctuation marks)
    and analysed text (lemmas and POS tags)
    """

    tokenized = ' '.join([token.text for token in nlp(sent) if (token.text.isalpha() or 
                                                                '-' in token.text or 
                                                                '—' in token.text)])
    lemmatized = []
    pos_tagged = [] 
    
    for token in tokenized.split():
        analysis = morph.parse(token)
        lemma, POS = parse_analysis(analysis)
        lemmatized.append(lemma)
        pos_tagged.append(POS)
    return tokenized, ' '.join(lemmatized), ' '.join(pos_tagged)

Создаем датасет: (ура)

In [545]:
def make_df(data):
    """
    create dataframe
    """
    df = []
    for article in tqdm(data, total=len(data)):
        title = article['title']
        link = 'https://ru.wikipedia.org/wiki/{}'.format('_'.join(quote(title).split()))
        for sent in article['res']:
            tokenized, lemmatized, pos_tagged = preprocess(sent)
            df.append((link, title, sent, tokenized, lemmatized, pos_tagged))
    return df

In [546]:
df = make_df(np.random.choice(np.array(df), 500, replace=False))

### Посмотрим что получилось

In [2]:
import pandas as pd
import numpy as np


df = pd.DataFrame(df, columns=['source', 'title', 'sentence', 'tokenized', 'lemmatized', 'pos'])

In [11]:
df

,Unnamed: 0,source,title,sentence,tokenized,lemmatized,pos
0,0,https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%...,Материальная точка,Материальная точка (частица) — обладающее мас...,Материальная точка частица — обладающее массой...,материальный точка частица — обладать масса те...,ADJ NOUN NOUN UNK PTCP NOUN NOUN NOUN NOUN NOU...
1,1,https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%...,Материальная точка,Является простейшей физической моделью в механ...,Является простейшей физической моделью в механике,являться простой физический модель в|в механик...,VERB ADJ ADJ NOUN NOUN|PREP NOUN|NOUN
2,2,https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%...,Материальная точка,Положение материальной точки в пространстве оп...,Положение материальной точки в пространстве оп...,положение материальный точка в|в пространство ...,NOUN ADJ NOUN NOUN|PREP NOUN VERB PART|CONJ|AD...
3,3,https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%...,Материальная точка,В классической механике масса материальной точ...,В классической механике масса материальной точ...,в|в классический механика|механик масса матери...,NOUN|PREP ADJ NOUN|NOUN NOUN ADJ NOUN VERB ADJ...
4,4,https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%...,Материальная точка,"Материальная точка — геометрическая точка, кот...",Материальная точка — геометрическая точка кото...,материальный точка — геометрический точка кото...,ADJ NOUN UNK ADJ NOUN ADJ PTCP NOUN|PREP NOUN ...
...,...,...,...,...,...,...,...
36827,36827,https://ru.wikipedia.org/wiki/1556%20%D0%B3%D0...,1556 год,"Антоний Сийский — преподобный Русской церкви, ...",Антоний Сийский — преподобный Русской церкви о...,антоний|антония сийский — преподобный русский ...,NOUN|NOUN ADJ UNK ADJ ADJ NOUN NOUN INTJ|NOUN|...
36828,36828,https://ru.wikipedia.org/wiki/1556%20%D0%B3%D0...,1556 год,"Физули — Классик азербайджанской поэзии, сыгра...",Физули — Классик азербайджанской поэзии сыграв...,физули — классик|классика азербайджанский поэз...,NOUN UNK NOUN|NOUN ADJ NOUN PTCP ADJ NOUN|NOUN...
36829,36829,https://ru.wikipedia.org/wiki/1556%20%D0%B3%D0...,1556 год,"Писал на азербайджанском, персидском и арабско...",Писал на азербайджанском персидском и арабском...,писать на|на|на азербайджанский персидский и|и...,VERB INTJ|PREP|PART ADJ ADJ INTJ|NOUN|PART|CON...
36830,36830,https://ru.wikipedia.org/wiki/1556%20%D0%B3%D0...,1556 год,"Хумаюн — второй из Великих Моголов, сын Бабура...",Хумаюн — второй из Великих Моголов сын Бабура ...,хумаюна — втора|второй иза|из великое|великий ...,NOUN UNK NOUN|ADJ NOUN|PREP NOUN|ADJ NOUN NOUN...


Еще немного пообрабатываем (спасибо пайморфи)

In [5]:
def get_rid_of_iza(sent):
    new = []
    for word in sent.split():
        if word == 'иза':
            word = 'из'
        new.append(word)
    return ' '.join(new)

In [12]:
df['lemmatized'] = df['lemmatized'].apply(get_rid_of_iza)

Ну и сохраним:

In [13]:
df.to_csv('corpora_data.csv', sep='\t')